In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def plot_dist(channel):
    fig, ax = plt.subplots(figsize=(4, 3))
    ax.hist(channel.ravel(), 25, [0,256])
    
    fig.canvas.draw()
    dist = np.array(fig.canvas.renderer.buffer_rgba())
    plt.close(fig) #Функция вывода стистики канала
    
    return dist

cv2.namedWindow("frame")

def nothing(x):
    pass

cv2.createTrackbar('hs', 'frame', 256, 512, nothing)
cv2.createTrackbar('ss', 'frame', 256, 512, nothing)
cv2.createTrackbar('vs', 'frame', 256, 512, nothing) #Создание трекбаров

video_path = "unicycle.mp4"
cam = cv2.VideoCapture(video_path)

while (True):
    success, frame = cam.read()
    
    if(success == False):
        cam.release()
        cam = cv2.VideoCapture(video_path)  
        
        continue #Проверка взятия кадра из видео
        
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    hs = cv2.getTrackbarPos('hs', 'frame')
    ss = cv2.getTrackbarPos('ss', 'frame')
    vs = cv2.getTrackbarPos('vs', 'frame') #Привязка трекбаров
    
    hsv_frame[:, :, 0] = cv2.add(hsv_frame[:, :, 0], hs - 256)
    hsv_frame[:, :, 1] = cv2.add(hsv_frame[:, :, 1], ss - 256)
    hsv_frame[:, :, 2] = cv2.add(hsv_frame[:, :, 2], vs - 256) #Изменение каналов с помощью трекбаров
    
    dist_0 = plot_dist(hsv_frame[::30, ::30, 0])
    dist_1 = plot_dist(hsv_frame[::30, ::30, 1])
    dist_2 = plot_dist(hsv_frame[::30, ::30, 2]) #Статистика по ограниченному количеству кадров картинки
    
    dists = np.concatenate((dist_0, dist_1, dist_2), axis=0) #Объединение статистик

    cv2.imshow('frame', cv2.cvtColor(hsv_frame, cv2.COLOR_HSV2BGR)) 
    cv2.imshow('dist_0', dists)
    
    key = cv2.waitKey(100) & 0xFF
    
    if (key == ord('q')):
        break

cam.release()
cv2.destroyAllWindows()
cv2.waitKey(10)

-1